<div dir=rtl>

## מעבדה לפיסיקה 2

</center>

<h1><center>התנגדות</center></h1>

<div dir=rtl>

תיעוד ניסוי "התנגדות"<br>
    <font color='red'>אין קובץ עזר זה מחליף את תדריך הניסוי! למהלך הניסוי, יש לפנות לתדריך הניסוי!</font>
    
    גרסה 0.15
    
***
<div dir=rtl>
תאריך:
<div dir=rtl>
עמדה:
<div dir=rtl>
מבצעים/ות:


***
<div dir=rtl> <b> הערות: </b> <br>
    .
    
.
    
.
    
.
    

<div dir=rtl>

### ייבוא חבילות לעיבוד הנתונים ולקיחת מדידות

In [1]:
import numpy as np # math functions
import matplotlib.pyplot as plt # for plotting figures and setting their properties
from scipy.integrate import cumtrapz #  numerical integration
from scipy.stats import linregress # linear regression

<div dir=rtl>
<h3>
   מתוך שאלות ההכנה:
</h3>

<div class="alert alert-block alert-danger", dir=rtl>
<b>טיפ:</b> נחוץ מקור לכל ציטוט ערך ספרותי
</div>

In [2]:
# Resistivity
rho_lit_Cu = # , Source:
rho_lit_Mo = # , Source:

# Resistance of a diam=0.07mm, length=20cm wire:
R_Cu = #
R_Mo = #

# Density
dens_lit_Cu = # , Source:
dens_lit_Mo = # , Source:

# Heat capacity
hc_lit_Cu = # , Source:
hc_lit_Mo = # , Source:

# Temperature coefficient of resistance (a)
alpha_lit_Cu = # , Source:
alpha_lit_Mo = # , Source:

SyntaxError: invalid syntax (4042567591.py, line 2)

<h1>
    מהלך הניסוי 
<a id="mahalah"></a>
</h1>

<h3>
   מדידות מקדימות:
</h3>

<div class="alert alert-block alert-info", dir=rtl>
<b>טיפ:</b> המולטימטר הוא הכלי היעודי שלנו למדידת התנגדות.
</div>

In [ ]:
# זכרו להשתמש במולטימטר בשביל למדוד את ההתנגדות
# High wattage resistor:
R_0 =
R_0_err = 

# Wire resistors:
# 
R_1 =
R_1_err = 
#
R_2 =
R_2_err = 
#
R_3 =
R_3_err = 
#
R_4 =
R_4_err = 


<div class="alert alert-block alert-warning", dir=rtl>
<b>בדיקה עצמית:</b> האם ההתנגדות המדודה תואמת לצפוי (לפי פירוט הנגד\מבנה התיל?)
</div>

<h3>
   מדידת פולס
</h3>


In [ ]:
def importData(filename):
  # use skiprows to skip the scope setup information
  # usecols column numbers starting from 0
  t, v1, v2 = np.loadtxt(filename, delimiter=',', skiprows=59, usecols=[3,4,5]).T
  return t, v1, v2

filename1 = ""
t1, Vx1, Vy1 = importData(filename1)
Vr1 = #
Ir1 = #

filename2 = ""
t2, Vx2, Vy2 = importData(filename2)
Vr2 = #
Ir2 = #

### plotting:



<div dir=rtl>
    <h3>
    מדידת התנגדות # (תיל: )
</h3>

<div class="alert alert-block alert-success", dir=rtl>
<b>רגע של מחשבה:</b> מה צפויות להיות יחידות הזמן? איך נרצה "לחתוך" את הצגת האות שלנו כדי להתמקד בעיקר?
</div>

In [ ]:
filename = ""
t, Vx, Vy = importData(filename)


### We can use indice slicing to focus only on the data we're interested in:
t0 = 0 # ms
t1 = 30e-3 # ms
interval = (t > t0) & (t < t1)
# t = t[interval]
# V = V[interval]

### plotting:



$$ \Delta Q(t) = \int_0^t I(t')V(t') dt' $$

$$ R(t) = R_{t=0}\left(1+\frac{a}{c\cdot m}\Delta Q(t)\right) $$

In [ ]:
### Given t, Vx, Vy, write a function that returns all the necessary data for analysis:
### Current through and Voltage on the wire (I, V), it's resistance (R) and accumulated energy (E)
def IVRE(t, Vx, Vy):
    
    return I,V,R,E


fit = linregress()

<div class="alert alert-block alert-danger", dir=rtl>
<b>שימו לב:</b> האם כל הקבצים (קבצי מדידות (csv), קבצי עיבוד, תמונות...) הועלו למודל?
</div>